In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange
import cv2
import os
import time

# ==========================================
# 1. KERR BLACK HOLE CONFIGURATION
# ==========================================

WIDTH = 640          # Resolution (Lowered slightly for speed, increase for final)
HEIGHT = 360
FOV = 50.0
SPIN_A = 0.99        # Black Hole Spin (0 to 1). 0.99 is near extremal.
RS = 1.0             # GM/c^2 = 1
# Event Horizon radius for Kerr: M + sqrt(M^2 - a^2)
R_HORIZON = RS + np.sqrt(RS**2 - SPIN_A**2) 

STEP_SIZE = 0.05
MAX_STEPS = 2000

# Animation Settings
NUM_FRAMES = 120     # Total frames for the loop
FPS = 24
OUTPUT_VIDEO = "kerr_black_hole_spin.mp4"
CAM_DIST = 18.0
INCLINATION_DEG = 85.0 # View nearly edge-on to see the D-shape shadow clearly

# Ensure output directory exists
os.makedirs("kerr_frames", exist_ok=True)

# ==========================================
# 2. NUMBA-ACCELERATED PHYSICS KERNELS
# ==========================================

@njit
def get_inverse_metric(pos):
    """
    Returns the inverse metric components g^uv at position (r, theta).
    Boyer-Lindquist coordinates.
    pos: [t, r, theta, phi]
    """
    r = pos[1]
    theta = pos[2]
    
    # Precompute trig and metric functions
    sin_th = np.sin(theta)
    cos_th = np.cos(theta)
    sigma = r**2 + (SPIN_A * cos_th)**2
    delta = r**2 - 2.0 * RS * r + SPIN_A**2
    
    # Inverse Metric Components (Non-zero only)
    # These are analytic solutions for the inverse Kerr metric
    # g^tt
    inv_g_tt = - ((r**2 + SPIN_A**2)**2 - delta * (SPIN_A * sin_th)**2) / (delta * sigma)
    
    # g^rr
    inv_g_rr = delta / sigma
    
    # g^thth
    inv_g_thth = 1.0 / sigma
    
    # g^phiphi
    # Avoid division by zero at poles
    sin_sq = sin_th**2
    if sin_sq < 1e-6: sin_sq = 1e-6
    inv_g_phiphi = (delta - SPIN_A**2 * sin_sq) / (delta * sigma * sin_sq)
    
    # g^tphi (The term responsible for frame dragging)
    inv_g_tphi = - (2.0 * RS * r * SPIN_A) / (delta * sigma)
    
    return inv_g_tt, inv_g_rr, inv_g_thth, inv_g_phiphi, inv_g_tphi

@njit
def hamiltonian(pos, p):
    """
    Computes H = 0.5 * g^uv * p_u * p_v
    """
    g_tt, g_rr, g_thth, g_ph, g_tph = get_inverse_metric(pos)
    
    H = 0.5 * (g_tt * p[0]**2 + 
               g_rr * p[1]**2 + 
               g_thth * p[2]**2 + 
               g_ph * p[3]**2 + 
               2.0 * g_tph * p[0] * p[3])
    return H

@njit
def get_derivatives(state):
    """
    Computes derivatives for the Hamiltonian system.
    state: [t, r, theta, phi, pt, pr, ptheta, pphi]
    Returns: [dt, dr, dth, dph, dpt, dpr, dpth, dpph]
    """
    pos = state[:4]
    p = state[4:]
    
    # 1. dx/dlambda = dH/dp (The velocity)
    # Analytically: x_dot^mu = g^mu_nu * p_nu
    g_tt, g_rr, g_thth, g_ph, g_tph = get_inverse_metric(pos)
    
    dt  = g_tt * p[0] + g_tph * p[3]
    dr  = g_rr * p[1]
    dth = g_thth * p[2]
    dph = g_ph * p[3] + g_tph * p[0]
    
    # 2. dp/dlambda = -dH/dx (The force/curvature)
    # We use numerical differentiation for the spatial gradients of the metric.
    # This avoids writing out the massive Christoffel symbols for Kerr.
    # We only need derivatives w.r.t r and theta (t and phi are cyclic/symmetric).
    
    epsilon = 1e-5
    
    # dH/dr
    pos_plus = np.array([pos[0], pos[1] + epsilon, pos[2], pos[3]])
    pos_minus = np.array([pos[0], pos[1] - epsilon, pos[2], pos[3]])
    dH_dr = (hamiltonian(pos_plus, p) - hamiltonian(pos_minus, p)) / (2.0 * epsilon)
    
    # dH/dtheta
    pos_plus[1] = pos[1] # Reset r
    pos_minus[1] = pos[1]
    pos_plus[2] = pos[2] + epsilon
    pos_minus[2] = pos[2] - epsilon
    dH_dth = (hamiltonian(pos_plus, p) - hamiltonian(pos_minus, p)) / (2.0 * epsilon)
    
    # dpt = -dH/dt = 0 (Stationary metric)
    dpt = 0.0
    # dpphi = -dH/dphi = 0 (Axisymmetric metric)
    dpphi = 0.0
    
    dpr = -dH_dr
    dpth = -dH_dth
    
    return np.array([dt, dr, dth, dph, dpt, dpr, dpth, dpphi])

@njit
def rk4_step_kerr(state, h):
    k1 = get_derivatives(state)
    k2 = get_derivatives(state + 0.5 * h * k1)
    k3 = get_derivatives(state + 0.5 * h * k2)
    k4 = get_derivatives(state + h * k3)
    return state + (h / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

@njit
def get_disk_color(r, phi, x_hit, y_hit, z_hit):
    """
    Procedural accretion disk texture.
    """
    # Create spiral/ring structure
    # Use global Cartesian coords for texture mapping to avoid polar singularity issues
    scale = 3.0
    spiral = np.sin(r * scale + np.arctan2(y_hit, x_hit))
    rings = np.sin(r * 5.0)
    
    brightness = 0.5 + 0.25 * spiral + 0.25 * rings
    
    # Doppler Beaming approximation
    # For Kerr, the disk rotates in the +phi direction.
    # Light coming from the side moving towards us is blueshifted/brighter.
    # x > 0 is roughly the side moving away, x < 0 moving towards (depending on cam)
    # Simple heuristic based on impact parameter:
    beaming = 1.0 - 0.6 * (x_hit / (r + 0.1)) 
    
    brightness *= beaming
    
    # Color mapping based on radius (temperature gradient)
    norm_r = (r - R_HORIZON) / 10.0
    col_r = 1.0 * brightness
    col_g = (0.3 + 0.5 * norm_r) * brightness
    col_b = (0.1 + 0.8 * norm_r) * brightness
    
    return np.array([col_r, col_g, col_b])

@njit
def ray_march_kerr(cam_pos_bl, ray_p_bl):
    """
    Traces a ray in Kerr metric.
    cam_pos_bl: Initial [t, r, theta, phi]
    ray_p_bl: Initial 4-momentum [pt, pr, ptheta, pphi]
    """
    state = np.concatenate((cam_pos_bl, ray_p_bl))
    
    # Normalize step size based on distance to avoid overstepping near horizon
    current_h = STEP_SIZE
    
    for i in range(MAX_STEPS):
        prev_r = state[1]
        prev_th = state[2]
        prev_phi = state[3] # Track phi for texture rotation if needed
        
        # Adaptive step size near horizon
        if state[1] < 3.0 * RS:
            current_h = STEP_SIZE * 0.2
        else:
            current_h = STEP_SIZE
            
        state = rk4_step_kerr(state, current_h)
        
        r = state[1]
        th = state[2]
        
        # 1. Horizon Check
        if r < R_HORIZON * 1.02:
            return np.array([0.0, 0.0, 0.0]) # Hit Black Hole
            
        # 2. Escape Check
        if r > CAM_DIST * 1.5:
             # Starfield background (simple noise)
             phi = state[3]
             stars = np.sin(phi * 20.0) * np.sin(th * 20.0)
             if stars > 0.98: return np.array([1.0, 1.0, 1.0])
             return np.array([0.0, 0.0, 0.05])
        
        # 3. Disk Intersection Check
        # Disk is at theta = pi/2 (equatorial plane)
        # We define a thin disk thickness
        if (prev_th - np.pi/2.0) * (th - np.pi/2.0) < 0.0:
            # We crossed the equator
            # Linear interpolation for exact intersection radius
            frac = abs(prev_th - np.pi/2.0) / (abs(prev_th - np.pi/2.0) + abs(th - np.pi/2.0))
            hit_r = prev_r + (r - prev_r) * frac
            hit_phi = prev_phi + (state[3] - prev_phi) * frac
            
            # Check limits of accretion disk
            if 3.0 * RS < hit_r < 15.0 * RS:
                # Convert back to Cartesian for texture utility
                x_h = hit_r * np.cos(hit_phi)
                y_h = hit_r * np.sin(hit_phi)
                z_h = 0.0
                return get_disk_color(hit_r, hit_phi, x_h, y_h, z_h)

    return np.array([0.0, 0.0, 0.0])

@njit(parallel=True)
def render_frame_data(width, height, cam_r, cam_theta, cam_phi, fov):
    """
    Renders the image data for a specific camera position.
    """
    image = np.zeros((height, width, 3))
    aspect = width / height
    
    # Screen plane vectors setup (Tetrad-like local flat space approximation)
    # At large distances, we can approximate the local frame as flat spherical
    
    # Camera basis vectors in Global Cartesian (for screen orientation)
    cx = cam_r * np.sin(cam_theta) * np.cos(cam_phi)
    cy = cam_r * np.sin(cam_theta) * np.sin(cam_phi)
    cz = cam_r * np.cos(cam_theta)
    cam_pos_cart = np.array([cx, cy, cz])
    
    # Look at origin
    fwd = -cam_pos_cart / np.linalg.norm(cam_pos_cart)
    # Global Up is Z
    glob_up = np.array([0.0, 0.0, 1.0])
    right = np.cross(fwd, glob_up)
    right = right / np.linalg.norm(right)
    up = np.cross(right, fwd)
    
    # Screen dimensions
    fov_rad = np.radians(fov)
    half_h = np.tan(fov_rad/2.0)
    half_w = half_h * aspect
    
    for y in prange(height):
        for x in prange(width):
            # Screen coord -1 to 1
            sx = (x / width) * 2.0 - 1.0
            sy = 1.0 - (y / height) * 2.0
            
            # Ray direction in local Cartesian frame
            ray_dir_cart = fwd + right * (sx * half_w) + up * (sy * half_h)
            ray_dir_cart = ray_dir_cart / np.linalg.norm(ray_dir_cart)
            
            # Convert Cartesian Position/Direction to Boyer-Lindquist Phase Space (x, p)
            # 1. Initial Position (BL): t=0, r=cam_r, th=cam_theta, ph=cam_phi
            init_pos = np.array([0.0, cam_r, cam_theta, cam_phi])
            
            # 2. Initial Momentum (BL): Convert Cartesian vector v to spherical velocity
            # v = vx*i + vy*j + vz*k
            vx, vy, vz = ray_dir_cart[0], ray_dir_cart[1], ray_dir_cart[2]
            
            # Standard Cartesian to Spherical Jacobian conversion for vector
            # (Assuming flat space metric at camera distance for initial condition approximation)
            # pr = (x*vx + y*vy + z*vz) / r
            pr = (cx*vx + cy*vy + cz*vz) / cam_r
            
            # ptheta = (1/r) * [ (xz*vx + yz*vy)/rho - rho*vz ] where rho = sqrt(x^2+y^2)
            # Standard spherical conversion formulas:
            rho = np.sqrt(cx**2 + cy**2)
            if rho < 1e-6:
                pth = 0.0 # Pole singularity check
                pph = 0.0
            else:
                pth = (cz * (cx*vx + cy*vy) - rho**2 * vz) / (cam_r * rho)  # Note: 1/r factor absorbed? No, p_theta is r*v_theta usually
                # Actually, p_theta corresponds to d/dtheta, so dimensions are momentum * length.
                # In orthonormal basis: p_hat_theta = (projection on theta_hat).
                # p_theta (covariant) = r * p_hat_theta
                
                # Let's project onto unit vectors first:
                # r_hat = (sin th cos ph, sin th sin ph, cos th)
                # th_hat = (cos th cos ph, cos th sin ph, -sin th)
                # ph_hat = (-sin ph, cos ph, 0)
                
                v_dot_rhat = vx*np.sin(cam_theta)*np.cos(cam_phi) + vy*np.sin(cam_theta)*np.sin(cam_phi) + vz*np.cos(cam_theta)
                v_dot_thhat = vx*np.cos(cam_theta)*np.cos(cam_phi) + vy*np.cos(cam_theta)*np.sin(cam_phi) - vz*np.sin(cam_theta)
                v_dot_phhat = -vx*np.sin(cam_phi) + vy*np.cos(cam_phi)
                
                # Convert physical velocity to conjugate momentum components
                # p_r = v_r
                # p_theta = r * v_theta
                # p_phi = r * sin(theta) * v_phi
                pr = v_dot_rhat
                pth = cam_r * v_dot_thhat
                pph = cam_r * np.sin(cam_theta) * v_dot_phhat
            
            # Energy E = -p_t. For photon, -p_t = |p|. We normalize spatial p.
            # E should be consistent with H=0.
            # Roughly p_t = -1.0 if we normalize ray energy.
            # More accurately, solve H(p_t) = 0 for p_t given pr, pth, pph.
            # In flat space limit at camera: -p_t = sqrt(pr^2 + pth^2/r^2 + pph^2/r^2sin^2th).
            # We already scaled pth and pph by r, so:
            pt = -1.0 
            
            init_p = np.array([pt, pr, pth, pph])
            
            image[y, x] = ray_march_kerr(init_pos, init_p)
            
    return image

# ==========================================
# 3. FRAME GENERATION & VIDEO COMPILATION
# ==========================================

def generate_video():
    print("Starting Kerr Black Hole Simulation...")
    print(f"Spin: {SPIN_A} (D-shaped shadow expected)")
    
    # 1. Compile Numba Functions (Warmup)
    print("Compiling JIT kernels (this takes 10-20s)...")
    _ = render_frame_data(10, 10, 20.0, 1.5, 0.0, 60.0)
    print("Compilation Complete.")
    
    # 2. Render Frames
    print(f"Rendering {NUM_FRAMES} frames...")
    
    inclination_rad = np.radians(INCLINATION_DEG)
    
    for i in range(NUM_FRAMES):
        filename = f"kerr_frames/frame_{i:03d}.png"
        
        # Resume capability
        if os.path.exists(filename):
            print(f"Frame {i} exists, skipping.")
            continue
            
        # Orbit logic: Rotating camera around phi
        phi = (2.0 * np.pi * i) / NUM_FRAMES
        
        t0 = time.time()
        img_data = render_frame_data(WIDTH, HEIGHT, CAM_DIST, inclination_rad, phi, FOV)
        
        # Convert to BGR for OpenCV
        img_data = np.clip(img_data, 0, 1) * 255.0
        img_bgr = img_data.astype(np.uint8)[..., ::-1] # Flip RGB to BGR
        
        cv2.imwrite(filename, img_bgr)
        print(f"Frame {i}/{NUM_FRAMES} rendered in {time.time()-t0:.2f}s")

    # 3. Compile Video
    print("Stitching video...")
    
    images = [img for img in sorted(os.listdir("kerr_frames")) if img.endswith(".png")]
    if not images:
        print("No images found to render.")
        return

    frame_path = os.path.join("kerr_frames", images[0])
    frame = cv2.imread(frame_path)
    h, w, layers = frame.shape

    video = cv2.VideoWriter(OUTPUT_VIDEO, cv2.VideoWriter_fourcc(*'mp4v'), FPS, (w, h))

    for image in images:
        video.write(cv2.imread(os.path.join("kerr_frames", image)))

    video.release()
    print(f"Video saved as {OUTPUT_VIDEO}")

if __name__ == "__main__":
    generate_video()

Starting Kerr Black Hole Simulation...
Spin: 0.99 (D-shaped shadow expected)
Compiling JIT kernels (this takes 10-20s)...
Compilation Complete.
Rendering 120 frames...
Frame 0/120 rendered in 17.44s
Frame 1/120 rendered in 17.53s
Frame 2/120 rendered in 17.03s
Frame 3/120 rendered in 17.54s
Frame 4/120 rendered in 16.99s
Frame 5/120 rendered in 18.03s
Frame 6/120 rendered in 17.57s
Frame 7/120 rendered in 17.98s
Frame 8/120 rendered in 17.55s
Frame 9/120 rendered in 17.63s
Frame 10/120 rendered in 17.68s
Frame 11/120 rendered in 17.93s
Frame 12/120 rendered in 17.30s
Frame 13/120 rendered in 17.72s
Frame 14/120 rendered in 17.30s
Frame 15/120 rendered in 17.35s
Frame 16/120 rendered in 17.58s
Frame 17/120 rendered in 16.94s
Frame 18/120 rendered in 18.71s
Frame 19/120 rendered in 17.72s
Frame 20/120 rendered in 16.88s
Frame 21/120 rendered in 17.61s
Frame 22/120 rendered in 16.87s
Frame 23/120 rendered in 17.43s
Frame 24/120 rendered in 16.94s
Frame 25/120 rendered in 16.97s
Frame 26/1

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange
import cv2
import os
import time

# ==========================================
# 1. HIGH-RES KERR CONFIGURATION
# ==========================================

# High Resolution Settings
WIDTH = 1000         
HEIGHT = 750
FOV = 48.0           # Slightly zoomed in for the larger canvas
SPIN_A = 0.99        # Near-extremal spin
RS = 1.0             
R_HORIZON = RS + np.sqrt(RS**2 - SPIN_A**2) 

# Integration Settings
STEP_SIZE = 0.05
MAX_STEPS = 2500     # Increased for high-res details

# Animation Settings
NUM_FRAMES = 360     # Smooth 360-degree loop
FPS = 30
OUTPUT_DIR = "Kerr_frames_HR"
OUTPUT_VIDEO = "kerr_black_hole_HR.mp4"
CAM_DIST = 33.0
INCLINATION_DEG = 85.0 

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==========================================
# 2. NUMBA-ACCELERATED PHYSICS KERNELS
# ==========================================

@njit
def get_inverse_metric(pos):
    """
    Returns the inverse metric components g^uv at position (r, theta).
    Boyer-Lindquist coordinates.
    """
    r = pos[1]
    theta = pos[2]
    
    sin_th = np.sin(theta)
    cos_th = np.cos(theta)
    sigma = r**2 + (SPIN_A * cos_th)**2
    delta = r**2 - 2.0 * RS * r + SPIN_A**2
    
    inv_g_tt = - ((r**2 + SPIN_A**2)**2 - delta * (SPIN_A * sin_th)**2) / (delta * sigma)
    inv_g_rr = delta / sigma
    inv_g_thth = 1.0 / sigma
    
    # Polar singularity protection
    sin_sq = sin_th**2
    if sin_sq < 1e-9: sin_sq = 1e-9
    
    inv_g_phiphi = (delta - SPIN_A**2 * sin_sq) / (delta * sigma * sin_sq)
    inv_g_tphi = - (2.0 * RS * r * SPIN_A) / (delta * sigma)
    
    return inv_g_tt, inv_g_rr, inv_g_thth, inv_g_phiphi, inv_g_tphi

@njit
def hamiltonian(pos, p):
    g_tt, g_rr, g_thth, g_ph, g_tph = get_inverse_metric(pos)
    H = 0.5 * (g_tt * p[0]**2 + 
               g_rr * p[1]**2 + 
               g_thth * p[2]**2 + 
               g_ph * p[3]**2 + 
               2.0 * g_tph * p[0] * p[3])
    return H

@njit
def get_derivatives(state):
    pos = state[:4]
    p = state[4:]
    g_tt, g_rr, g_thth, g_ph, g_tph = get_inverse_metric(pos)
    
    dt  = g_tt * p[0] + g_tph * p[3]
    dr  = g_rr * p[1]
    dth = g_thth * p[2]
    dph = g_ph * p[3] + g_tph * p[0]
    
    epsilon = 1e-5
    
    # dH/dr
    pos_plus = np.array([pos[0], pos[1] + epsilon, pos[2], pos[3]])
    pos_minus = np.array([pos[0], pos[1] - epsilon, pos[2], pos[3]])
    dH_dr = (hamiltonian(pos_plus, p) - hamiltonian(pos_minus, p)) / (2.0 * epsilon)
    
    # dH/dtheta
    pos_plus[1] = pos[1]
    pos_minus[1] = pos[1]
    pos_plus[2] = pos[2] + epsilon
    pos_minus[2] = pos[2] - epsilon
    dH_dth = (hamiltonian(pos_plus, p) - hamiltonian(pos_minus, p)) / (2.0 * epsilon)
    
    dpt = 0.0
    dpphi = 0.0
    dpr = -dH_dr
    dpth = -dH_dth
    
    return np.array([dt, dr, dth, dph, dpt, dpr, dpth, dpphi])

@njit
def rk4_step_kerr(state, h):
    k1 = get_derivatives(state)
    k2 = get_derivatives(state + 0.5 * h * k1)
    k3 = get_derivatives(state + 0.5 * h * k2)
    k4 = get_derivatives(state + h * k3)
    return state + (h / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

@njit
def get_disk_color(r, phi, x_hit, y_hit, z_hit):
    """
    Procedural accretion disk texture.
    """
    scale = 3.0
    # Use global Cartesian coords for texture mapping to avoid polar singularity issues in phi
    # This prevents texture seams even if phi jumps by 2pi
    spiral = np.sin(r * scale + np.arctan2(y_hit, x_hit))
    rings = np.sin(r * 5.0)
    
    brightness = 0.5 + 0.25 * spiral + 0.25 * rings
    beaming = 1.0 - 0.6 * (x_hit / (r + 0.1)) 
    brightness *= beaming
    
    norm_r = (r - R_HORIZON) / 10.0
    col_r = 1.0 * brightness
    col_g = (0.3 + 0.5 * norm_r) * brightness
    col_b = (0.1 + 0.8 * norm_r) * brightness
    
    return np.array([col_r, col_g, col_b])

@njit
def ray_march_kerr(cam_pos_bl, ray_p_bl):
    state = np.concatenate((cam_pos_bl, ray_p_bl))
    
    for i in range(MAX_STEPS):
        prev_r = state[1]
        prev_th = state[2]
        prev_phi = state[3]
        
        # --- ADAPTIVE STEP SIZE (THE FIX) ---
        # 1. Slow down near Event Horizon
        if state[1] < 2.0 * RS:
            h_factor = 0.2
        # 2. Slow down near Polar Singularity (theta ~ 0 or pi)
        # This fixes the vertical artifact at 12 o'clock
        elif np.abs(np.sin(state[2])) < 0.1: 
            h_factor = 0.1
        else:
            h_factor = 1.0
            
        current_h = STEP_SIZE * h_factor
            
        state = rk4_step_kerr(state, current_h)
        
        r = state[1]
        th = state[2]
        
        # Horizon Check
        if r < R_HORIZON * 1.02:
            return np.array([0.0, 0.0, 0.0])
            
        # Escape Check
        if r > CAM_DIST * 1.5:
             # Starfield
             stars = np.sin(state[3] * 20.0) * np.sin(th * 20.0)
             if stars > 0.98: return np.array([1.0, 1.0, 1.0])
             return np.array([0.0, 0.0, 0.05])
        
        # Disk Intersection Check (Equatorial Plane)
        if (prev_th - np.pi/2.0) * (th - np.pi/2.0) < 0.0:
            frac = abs(prev_th - np.pi/2.0) / (abs(prev_th - np.pi/2.0) + abs(th - np.pi/2.0))
            hit_r = prev_r + (r - prev_r) * frac
            
            # Interpolate Phi safely accounting for 2pi wrapping if necessary
            # (Though small steps near pole usually prevent massive wrapping issues)
            hit_phi = prev_phi + (state[3] - prev_phi) * frac
            
            if 3.0 * RS < hit_r < 15.0 * RS:
                x_h = hit_r * np.cos(hit_phi)
                y_h = hit_r * np.sin(hit_phi)
                return get_disk_color(hit_r, hit_phi, x_h, y_h, 0.0)

    return np.array([0.0, 0.0, 0.0])

@njit(parallel=True)
def render_frame_data(width, height, cam_r, cam_theta, cam_phi, fov):
    image = np.zeros((height, width, 3))
    aspect = width / height
    
    cx = cam_r * np.sin(cam_theta) * np.cos(cam_phi)
    cy = cam_r * np.sin(cam_theta) * np.sin(cam_phi)
    cz = cam_r * np.cos(cam_theta)
    cam_pos_cart = np.array([cx, cy, cz])
    
    fwd = -cam_pos_cart / np.linalg.norm(cam_pos_cart)
    glob_up = np.array([0.0, 0.0, 1.0])
    right = np.cross(fwd, glob_up)
    right = right / np.linalg.norm(right)
    up = np.cross(right, fwd)
    
    fov_rad = np.radians(fov)
    half_h = np.tan(fov_rad/2.0)
    half_w = half_h * aspect
    
    for y in prange(height):
        for x in prange(width):
            sx = (x / width) * 2.0 - 1.0
            sy = 1.0 - (y / height) * 2.0
            
            ray_dir_cart = fwd + right * (sx * half_w) + up * (sy * half_h)
            ray_dir_cart = ray_dir_cart / np.linalg.norm(ray_dir_cart)
            
            init_pos = np.array([0.0, cam_r, cam_theta, cam_phi])
            
            vx, vy, vz = ray_dir_cart[0], ray_dir_cart[1], ray_dir_cart[2]
            pr = (cx*vx + cy*vy + cz*vz) / cam_r
            
            rho = np.sqrt(cx**2 + cy**2)
            if rho < 1e-6:
                pth = 0.0
                pph = 0.0
            else:
                v_dot_thhat = vx*np.cos(cam_theta)*np.cos(cam_phi) + vy*np.cos(cam_theta)*np.sin(cam_phi) - vz*np.sin(cam_theta)
                v_dot_phhat = -vx*np.sin(cam_phi) + vy*np.cos(cam_phi)
                pth = cam_r * v_dot_thhat
                pph = cam_r * np.sin(cam_theta) * v_dot_phhat
            
            init_p = np.array([-1.0, pr, pth, pph])
            image[y, x] = ray_march_kerr(init_pos, init_p)
            
    return image

# ==========================================
# 3. MAIN EXECUTION
# ==========================================

def main():
    print("Initializing High-Res Kerr Simulation...")
    print(f"Resolution: {WIDTH}x{HEIGHT}")
    print(f"Frames: {NUM_FRAMES}")
    print(f"Output: {OUTPUT_DIR}/")
    
    # Compile
    print("Compiling JIT kernels (warmup)...")
    _ = render_frame_data(10, 10, 20.0, 1.5, 0.0, 60.0)
    print("Compilation Complete.")
    
    inclination_rad = np.radians(INCLINATION_DEG)
    
    print("Rendering frames...")
    for i in range(NUM_FRAMES):
        filename = f"{OUTPUT_DIR}/frame_{i:03d}.png"
        
        if os.path.exists(filename):
            continue
            
        phi = (2.0 * np.pi * i) / NUM_FRAMES
        
        t0 = time.time()
        img_data = render_frame_data(WIDTH, HEIGHT, CAM_DIST, inclination_rad, phi, FOV)
        
        img_data = np.clip(img_data, 0, 1) * 255.0
        img_bgr = img_data.astype(np.uint8)[..., ::-1]
        
        cv2.imwrite(filename, img_bgr)
        
        # Calculate ETA
        elapsed = time.time() - t0
        remaining = (NUM_FRAMES - i - 1) * elapsed
        print(f"Frame {i+1}/{NUM_FRAMES} | {elapsed:.2f}s | ETA: {remaining/60:.1f} min")

    print("Stitching video...")
    images = [img for img in sorted(os.listdir(OUTPUT_DIR)) if img.endswith(".png")]
    if not images: return

    frame_path = os.path.join(OUTPUT_DIR, images[0])
    frame = cv2.imread(frame_path)
    h, w, _ = frame.shape

    video = cv2.VideoWriter(OUTPUT_VIDEO, cv2.VideoWriter_fourcc(*'mp4v'), FPS, (w, h))

    for image in images:
        video.write(cv2.imread(os.path.join(OUTPUT_DIR, image)))

    video.release()
    print(f"Done! Video saved to {OUTPUT_VIDEO}")

if __name__ == "__main__":
    main()

Initializing High-Res Kerr Simulation...
Resolution: 1000x750
Frames: 360
Output: Kerr_frames_HR/
Compiling JIT kernels (warmup)...
Compilation Complete.
Rendering frames...
Frame 1/360 | 104.62s | ETA: 626.0 min
Frame 2/360 | 105.75s | ETA: 631.0 min
Frame 3/360 | 103.79s | ETA: 617.6 min
Frame 4/360 | 103.61s | ETA: 614.7 min
Frame 5/360 | 104.60s | ETA: 618.9 min
Frame 6/360 | 102.57s | ETA: 605.1 min
Frame 7/360 | 103.93s | ETA: 611.5 min
Frame 8/360 | 102.93s | ETA: 603.8 min
Frame 9/360 | 104.71s | ETA: 612.6 min
Frame 10/360 | 110.66s | ETA: 645.5 min
Frame 11/360 | 106.10s | ETA: 617.1 min
Frame 12/360 | 105.08s | ETA: 609.4 min
Frame 13/360 | 104.93s | ETA: 606.8 min
Frame 14/360 | 106.43s | ETA: 613.7 min
Frame 15/360 | 105.95s | ETA: 609.2 min
Frame 16/360 | 103.91s | ETA: 595.8 min
Frame 17/360 | 107.17s | ETA: 612.7 min
Frame 18/360 | 113.26s | ETA: 645.6 min
Frame 19/360 | 110.94s | ETA: 630.5 min
Frame 20/360 | 104.56s | ETA: 592.5 min
Frame 21/360 | 111.57s | ETA: 630.4